# Проект большие языковые модели

### Подготовка окружения

In [1]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.normalizers import NFD, Lowercase, StripAccents, Sequence
from tokenizers.processors import TemplateProcessing
from transformers import PreTrainedTokenizerFast, LlamaConfig, LlamaForCausalLM, get_linear_schedule_with_warmup
from torch.optim import AdamW 
import os
import gc
import torch
import warnings
from torch.utils.data import DataLoader
import regex as re
import numpy as np
import random
from pathlib import Path
from datasets import Dataset
import unicodedata
from collections import Counter
from tqdm.notebook import tqdm

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = True

seed_everything(42)
warnings.filterwarnings('ignore')

In [3]:
DATA_DIR = Path("RussianNovels/corpus")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
VOCAB_SIZE = 3000
CYRILLIC_RE = re.compile(r"^[\p{IsCyrillic}\s0-9.,!?—–:;\"'()«»…\-]+$")
BATCH_SIZE = 12
MAX_EPOCHS = 10
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 0.01
GRAD_CLIP = 1.0
SAVE_PATH = "best_model.pt"

test_prompts = [
    "Все мысли, которые имеют огромные последствия",
    "Сила войска зависит от его духа",
    "Мысль о том, что он принес страдания",
    "Человек сознает себя свободным",
    "Что бы ни случилось, я всегда буду",
    "Любовь мешает смерти",
    "Нет, жизнь не кончена",
    "Всякая мысль, даже самая простая",
    "Война не любезность, а самое гадкое дело",
    "Чтобы жить честно"
] 

print(f"{DEVICE=}")

DEVICE='cuda'


In [4]:
!git clone https://github.com/JoannaBy/RussianNovels.git
!git clone https://huggingface.co/datasets/d0rj/alpaca-cleaned-ru

fatal: destination path 'RussianNovels' already exists and is not an empty directory.
fatal: destination path 'alpaca-cleaned-ru' already exists and is not an empty directory.


## Pretrain

### Подготовка корпуска текстов

In [5]:
def collect_texts(root: Path):
    texts = []
    for path in root.rglob("*"):
        if path.suffix in {".txt"}:
            try:
                texts.append(path.read_text(encoding="utf-8"))
            except Exception:
                pass
    return texts

raw_texts = collect_texts(DATA_DIR)
raw_texts = list(set(raw_texts))
print(f"Файлов в корпусе: {len(raw_texts)}")

Файлов в корпусе: 107


In [6]:
def filter_cyrillic_sentences(text: str):
    '''фильтруем предложения с некириллическими символами'''
    sentences = re.split(r"[.!?]+", text)
    return [
        s.strip()
        for s in sentences
        if s.strip() and CYRILLIC_RE.match(s.strip())
    ]

def normalize_punctuation(text: str) -> str:
    '''нормализуемы пунктуацию'''
    text = re.sub(r"[!?]{2,}", r"\1", text)
    text = re.sub(r"\.{3,}", "…", text)
    text = re.sub(r",,{2,}", ",", text)
    text = re.sub(r"-{2,}", "-", text)
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r'[«»]', '"', text)
    return text.strip()    

clean_sentences = []

for text in raw_texts:
    sentences = filter_cyrillic_sentences(text)
    sentences = [normalize_punctuation(s) for s in sentences]
    clean_sentences.extend(sentences)

print(f"Предложений после очистки: {len(clean_sentences)}")

Предложений после очистки: 523962


In [7]:
print("Проверка состава оставшихся спец символов")
def is_letter_or_digit(ch: str) -> bool:
    cat = unicodedata.category(ch)
    return cat.startswith("L") or cat.startswith("N")

counter = Counter()

for sent in clean_sentences:
    for ch in sent:
        if not is_letter_or_digit(ch):
            counter[ch] += 1


non_alnum = counter.most_common()
print(f"symb | freg    | name")
for ch, freq in non_alnum[:30]:
    name = unicodedata.name(ch, "UNKNOWN")
    print(f"'{ch}'  | {freq:>7} | {name}")

Проверка состава оставшихся спец символов
symb | freg    | name
' '  | 5716309 | SPACE
','  |  853248 | COMMA
'-'  |  279089 | HYPHEN-MINUS
'"'  |   49998 | QUOTATION MARK
':'  |   44922 | COLON
';'  |   32563 | SEMICOLON
'–'  |   24641 | EN DASH
')'  |    7897 | RIGHT PARENTHESIS
'('  |    7337 | LEFT PARENTHESIS
'—'  |    4435 | EM DASH
'…'  |    2988 | HORIZONTAL ELLIPSIS
'''  |    1152 | APOSTROPHE


In [8]:
# разбиение на чанки
chunks = []
current = ""

for sent in clean_sentences:
    max_chars = 2000  # ~512 токенов
    if len(current) + len(sent) < max_chars:
        current += " " + sent
    else:
        chunks.append(current.strip())
        current = sent

if current:
    chunks.append(current.strip())

print(f"Чанков: {len(chunks)}")
print(f"чанк  0: {chunks[0]}")
print(f"чанк 10: {chunks[10]}")


Чанков: 20039
чанк  0: ) объявили смертный приговор шепотом Все встали, обмениваясь улыбками Седой судья, припав к его уху, подышав, сообщив, медленно отодвинулся, как будто отлипал Засим Цинцинната отвезли обратно в крепость Дорога обвивалась вокруг ее скалистого подножья и уходила под ворота: змея в расселину Был спокоен; однако его поддерживали во время путешествия по длинным коридорам, ибо он неверно ставил ноги, вроде ребенка, только что научившегося ступать, или точно куда проваливался, как человек, во сне увидевший, что идет по воде, но вдруг усомнившийся: да можно ли Тюремщик Родион долго отпирал дверь Цинциннатовой камеры, - не тот ключ, - всегдашняя возня Дверь наконец уступила Там, на койке, уже ждал адвокат - сидел, погруженный по плечи в раздумье, без фрака (забытого на венском стуле в зале суда, - был жаркий, насквозь синий день), - и нетерпеливо вскочил, когда ввели узника Но Цинциннату было не до разговоров Пускай одиночество в камере с глазком подобно ладье, дающей теч

### Обучение токенизатора

In [9]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

# Нормализация текста
tokenizer.normalizer = Sequence([NFD(), Lowercase(), StripAccents()])
# разбиение по пробелам 
tokenizer.pre_tokenizer = Whitespace() 
trainer = BpeTrainer(
    vocab_size=VOCAB_SIZE,
    min_frequency=2,
    special_tokens=["[PAD]", "[UNK]", "[BOS]", "[EOS]"] 
    )
tokenizer.train_from_iterator(chunks, trainer)
# (BOS/EOS) 
tokenizer.post_processor = TemplateProcessing(
    single="[BOS] $A [EOS]",
    pair="[BOS] $A [EOS] $B:1 [EOS]:1",
    special_tokens=[
        ("[BOS]", tokenizer.token_to_id("[BOS]")),
        ("[EOS]", tokenizer.token_to_id("[EOS]")), 
        ], 
        )
# Сохранение
tokenizer.save("bpe_ru_3k.json")

In [10]:
encoded = tokenizer.encode("Пример текста для проверки токенизатора")
print(encoded.tokens)
print(encoded.ids)


['[BOS]', 'пример', 'те', 'к', 'ста', 'для', 'про', 'вер', 'ки', 'то', 'ке', 'ни', 'за', 'тора', '[EOS]']
[2, 1903, 80, 31, 123, 308, 106, 170, 100, 58, 220, 70, 81, 1680, 3]


### Сборка датасета

In [11]:

hf_tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="bpe_ru_3k.json",
    bos_token="[BOS]",
    eos_token="[EOS]",
    unk_token="[UNK]",
    pad_token="[PAD]",
)

def tokenize(batch):
    return hf_tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

def add_labels(batch):
    labels = []
    for ids, mask in zip(batch["input_ids"], batch["attention_mask"]):
        l = ids.copy()
        l = [
            token if m == 1 else -100
            for token, m in zip(l, mask)
        ]
        labels.append(l)

    batch["labels"] = labels
    return batch


dataset = Dataset.from_dict({"text": chunks})

dataset = dataset.map(
    tokenize,
    batched=True,
    remove_columns=["text"],
).map(add_labels, batched=True)

dataset.set_format(type="torch")

Map:   0%|          | 0/20039 [00:00<?, ? examples/s]

Map:   0%|          | 0/20039 [00:00<?, ? examples/s]

### Подготовка модели LlamaForCausalLM

In [12]:
config = LlamaConfig(
    hidden_size=1024,
    intermediate_size=1536,
    num_hidden_layers=16,
    num_attention_heads=16,
    num_key_value_heads=8,
    max_position_embeddings=2048,
    rms_norm_eps=1e-6,
    hidden_act="silu",
    tie_word_embeddings=True,
    attention_bias=False,
    rope_theta=10000.0,
    use_cache=False,     
)
config.vocab_size = max(hf_tokenizer.get_vocab().values()) + 1
config.pad_token_id = hf_tokenizer.pad_token_id
config.bos_token_id = hf_tokenizer.bos_token_id
config.eos_token_id = hf_tokenizer.eos_token_id

model = LlamaForCausalLM(config).to(DEVICE)
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Обучаемых параметров: {num_params:,}")

Обучаемых параметров: 128,934,912


In [13]:
print("Проверим синхронизацию модели и текенизатора")
assert model.config.vocab_size == max(hf_tokenizer.get_vocab().values()) + 1
assert model.config.pad_token_id == hf_tokenizer.pad_token_id
assert model.config.eos_token_id == hf_tokenizer.eos_token_id
print("ок")

Проверим синхронизацию модели и текенизатора
ок


In [14]:
print("Sanity-check начальной модели на инференсе")
model.eval()

prompt = "Привет"
inputs = hf_tokenizer(
    prompt,
    return_tensors="pt",
    return_token_type_ids=False
).to(model.device)

with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=30,
        do_sample=False,
        eos_token_id=hf_tokenizer.eos_token_id,
    )

print(hf_tokenizer.decode(out[0], skip_special_tokens=True))


Sanity-check начальной модели на инференсе
при вет кры кры брат брат брат брат брат брат брат брат бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары бары


In [15]:
# оптимизатор
def get_optim():
    return AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

optimizer = get_optim()

# освобождение памяти
def gc_collect():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize() 

In [16]:
print("Сheck сходимости модели при обучение на малом датасете (2 предожения)")
tiny_dataset = dataset.select(range(2))
tiny_loader = DataLoader(tiny_dataset, batch_size=1, shuffle=True)

model.train()

progress_bar = tqdm(
        range(201),
        desc=f"train check",
    )
best_loss = float("inf")

for step in progress_bar:
    batch = next(iter(tiny_loader))
    batch = {k: v.to(model.device) for k, v in batch.items()}

    loss = model(**batch).loss
    progress_bar.set_description(f"train check | loss: {loss:.2f}")
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if step> 0 and step % 20 == 0:
        progress_bar.write(f"epoch: {step:>3} loss: {loss.item():.2f}")
        if loss.item() < best_loss:
            best_loss = loss.item()
            torch.save(model.state_dict(), SAVE_PATH)
            progress_bar.write(f"best model saved with {best_loss=:.2f}")

del tiny_dataset, tiny_loader, optimizer
gc_collect()

Сheck сходимости модели при обучение на малом датасете (2 предожения)


train check:   0%|          | 0/201 [00:00<?, ?it/s]

epoch:  20 loss: 8.74
best model saved with best_loss=8.74
epoch:  40 loss: 5.00
best model saved with best_loss=5.00
epoch:  60 loss: 5.15
epoch:  80 loss: 4.00
best model saved with best_loss=4.00
epoch: 100 loss: 2.69
best model saved with best_loss=2.69
epoch: 120 loss: 0.52
best model saved with best_loss=0.52
epoch: 140 loss: 0.13
best model saved with best_loss=0.13
epoch: 160 loss: 0.05
best model saved with best_loss=0.05
epoch: 180 loss: 0.02
best model saved with best_loss=0.02
epoch: 200 loss: 0.01
best model saved with best_loss=0.01


In [17]:
print("Сheck инференса модели после обучения на малом датасете")
def check_inference(model, prompts: list):
    model.eval()
    for pr in prompts:
        inputs = hf_tokenizer(pr, return_tensors="pt").to(model.device)
        inputs.pop("token_type_ids", None)

        out = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=True, 
        eos_token_id=hf_tokenizer.eos_token_id,
    )
        print("input:  ",  pr)
        print("output: ", hf_tokenizer.decode(out[0], skip_special_tokens=True))
        print()


check_inference(model, test_prompts[:2])

Сheck инференса модели после обучения на малом датасете
input:   Все мысли, которые имеют огромные последствия
output:  все мысли , которые име ют огром ные послед ствия ли ли все все все все все все все я де , су дья , как у я у ху , под в ы ша в , сообщи в , медленно нулся , как в нулся , за си м цин цин цин ната отве зли обра тно в кре

input:   Сила войска зависит от его духа
output:  сила вои ска зави си т от его ду ха объя ли все все все все все все все все все все все пред ками б ками плечи су су его при пав к , к его в , медленно ото дви нулся , медленно нулся , как м м цин цин ната отве зли обра тно в кре по



### Обучение модели LlamaForCausalLM

In [18]:
train_loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=True
)
model.train()

optimizer = get_optim()

total_steps = len(train_loader) * MAX_EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=total_steps // 10, 
    num_training_steps=total_steps
)

best_loss = float("inf")

for epoch in range(MAX_EPOCHS):

    progress_bar = tqdm(train_loader)

    total_loss = 0.0
    for step, batch in enumerate(progress_bar):
        batch = {k: v.to(DEVICE) for k, v in batch.items()}

        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"]
        )
        loss = outputs.loss

        progress_bar.set_description(
            f"train epoch {str(epoch+1):<2} step {str(step+1):<3} loss: {loss.item():.2f}"
        )
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)

        # Optimizer step
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        if step>0 and step % 100 == 0:
            progress_bar.write(f"epoch {epoch+1} step {step+1} loss: {loss.item():.2f}")

    avg_loss = total_loss / len(train_loader)

    progress_bar.write(f"epoch {epoch+1} finished. avg_loss: {avg_loss:.2f}")
    
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), SAVE_PATH)
        progress_bar.write(f"best model saved with {best_loss=:.2f}")

  0%|          | 0/1670 [00:00<?, ?it/s]

epoch 1 step 101 loss: 6.61
epoch 1 step 201 loss: 6.34
epoch 1 step 301 loss: 5.97
epoch 1 step 401 loss: 5.69
epoch 1 step 501 loss: 5.54
epoch 1 step 601 loss: 5.20
epoch 1 step 701 loss: 5.24
epoch 1 step 801 loss: 5.14
epoch 1 step 901 loss: 5.06
epoch 1 step 1001 loss: 4.84


KeyboardInterrupt: 

In [ ]:
from transformers import Trainer, TrainingArguments

split_dataset = dataset.train_test_split(test_size=0.1)

train_dataset = split_dataset['train']
val_dataset = split_dataset['test']

training_args = TrainingArguments(
    output_dir="./checkpoints",
    num_train_epochs=2,
    learning_rate=LEARNING_RATE,

    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=16,  # 16 × 8 = 128
    weight_decay=WEIGHT_DECAY,
    eval_strategy="steps",
    save_strategy="no",
    eval_steps=10,
    logging_steps=11,
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    # callbacks=[
    #     PromptValidationCallback(
    #         tokenizer=tokenizer,
    #         prompts=[
    #             "Translate to English: Привет мир",
    #             "Solve: 2 + 2 = ?"
    #         ],
    #         expected_outputs=[
    #             "Hello world",
    #             "4"
    #         ]
    #     )
    # ]
)
trainer.train()

Step,Training Loss,Validation Loss


In [27]:
gc_collect()